In [ ]:
import pandas as pd
import graph_ruggedness_de
import matplotlib.pyplot as plt 
import networkx as nx
import numpy as np
import sequence_evolution
from ete3 import Tree
import ete3
import random
import pyvolve
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

### Sequence evolution simulation
The below cell simulates the simulation of amino acid sequences, drawing parameters from uniform distributions of realistic values. The cell has also been converted to functional code in `src.sequence_evolution.sample_sequences`. The output is a dictionary of Dirichlet energies over the first 50 eigenvectors of the simulated graph Laplacian, simulating the fitness function as increasingly rugged.

In [ ]:
dir_dict = {}
prob_dict = {}
sample_dict = {}

sample_size = 100
for _ in tqdm(enumerate(range(sample_size))):
    try:

        num_nodes = int(np.random.uniform(50, 200))
        mean_branch_length = np.random.uniform(0.01, 0.5)
        std_dev_branch_length = np.random.uniform(0.001, 0.1)
        alpha = np.random.uniform(0.1, 0.5)
        model = random.choice(('WAG', 'LG'))
        rate_categoies = 4
        sequence_length = int(np.random.uniform(100, 600))

        sample_info = {
            'num_nodes' : num_nodes,
            'mean_branch_length' : mean_branch_length,
            'std_dev_branch_length' : std_dev_branch_length,
            'alpha' : alpha,
            'model' : model,
            'sequence_length' : sequence_length
        }

        seq_dict = sequence_evolution.sequence_evolution(num_nodes=num_nodes,
                                                        mean_branch_length=mean_branch_length,
                                                        std_dev_branch_length=std_dev_branch_length,
                                                        alpha=alpha,
                                                        model=model,
                                                        rate_categories=rate_categoies,
                                                        sequence_length=sequence_length
                                                        )
        seq_ls = list(seq_dict.values())
        values = [0]*len(seq_ls)
        G = graph_ruggedness_de.build_ohe_graph(seq_ls=seq_ls,
                                            values=values)
        eign_prob_dict = {}
        eign_dir_dict = {}
        for eign in list(range(50)):
            G = graph_ruggedness_de.compute_elementary_landscape(G=G, n=eign)
            dir_energy = graph_ruggedness_de.compute_dirichlet_energy(G=G)
            eign_dir_dict[eign] = dir_energy
        
        dir_dict[_] = eign_dir_dict
        prob_dict[_] = eign_prob_dict
        sample_dict[_] = sample_info
    
    except:
        continue


### Dirichlet energy distribution
The below cell plots the disitrubtion of Dirichlet energies over different randomly sampled evolutionary simulations as a function of the Laplacian eigenvector index. 

In [ ]:
rows = []
for i in dir_dict.keys():
    rows.append(dir_dict[i])
df_dir = pd.DataFrame(rows)

plt.figure(figsize=(8, 3))
box = df_dir.boxplot(patch_artist=True, color='grey')

ticks = np.arange(1, len(df_dir.columns) + 1, 10)
tick_labels = [df_dir.columns[i-1] for i in ticks]
plt.xticks(ticks, tick_labels, rotation=90)

plt.grid(visible=False)
box.spines['top'].set_visible(False)
box.spines['right'].set_visible(False)
plt.xlabel('Laplacian eigenvector')
plt.ylabel('Dirichlet energy')
plt.tight_layout()
plt.savefig('figures/Figure_1/Figure_1_boxplot.pdf')
plt.show()


### Dirichlet energy is correlated with graph size
The below cell plots the Dirichlet energy of the graph as a function of the Laplacian eigenvector index, and colours each simulation as a trace according to the size of the graph. 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import numpy as np

num_nodes_values = [sample['num_nodes'] for sample in sample_dict.values()]
min_num_nodes, max_num_nodes = min(num_nodes_values), max(num_nodes_values)

norm = mcolors.Normalize(vmin=min_num_nodes, vmax=max_num_nodes)
cmap = cm.plasma

plt.figure(figsize=(4, 2))

for key in dir_dict.keys():
    vals = list(dir_dict[key].values())
    num_nodes = sample_dict[key]['num_nodes']
    color = cmap(norm(num_nodes))
    

    plt.plot(vals, alpha=0.75, linewidth=1, linestyle=':', color=color)

plt.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), label='Number of Nodes')
plt.xlabel('Laplacian eigenvector')
plt.xticks(rotation=90)
plt.ylabel('Dirichlet energy')
plt.tight_layout()

ax = plt.gca() 
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.xaxis.tick_bottom()
ax.yaxis.tick_left()

plt.savefig('figures/Figure_1/Figure_1_trace.pdf')
plt.show()


### Graph structure and fitness distribution
The below cell plots graphs for with simulated fitness signals as the first and 50th eigenvectors of the same graph Laplacian. The includes both a computation of the dirichlet energy and the distribution of fitnesses. 

In [ ]:
seq_ls = list(seq_dict.values())
values = [0]*len(seq_ls)
G = graph_ruggedness_de.build_ohe_graph(seq_ls=seq_ls,
                                    values=values)

G_elem = graph_ruggedness_de.compute_elementary_landscape(G,
                                                 1)
pos = nx.spring_layout(G)
dir_energy = graph_ruggedness_de.compute_dirichlet_energy(G=G_elem)
values = [node[1]['value'] for node in G_elem.nodes(data=True)]
viridis = plt.cm.get_cmap('viridis', 10)
node_colors = [viridis((value - min(values)) / (max(values) - min(values))) for value in values]
nx.draw(G_elem, pos, node_color=node_colors, with_labels=False, edgecolors='black', node_size=100, width=0.75, edge_color='grey')
plt.savefig('figures/Figure_1/0th_eigvector_graph.pdf')
plt.show()

plt.figure(figsize=(1.05, 1.5))
plt.bar(x=0, height=dir_energy, color='grey')
plt.xticks([])
plt.ylim(0, 15)
plt.ylabel('Dirichlet Energy')
plt.tight_layout()
plt.savefig('figures/Figure_1/eigenvector_1_DE.pdf')
plt.show()

plt.figure(figsize=(2.5, 1.5))
plt.hist(values, bins=22, color='grey')
plt.xlabel('Simulated fitness')
plt.ylabel('Frequency')
plt.tight_layout()
plt.savefig('figures/Figure_1/eigenvector_1_hist.pdf')
plt.show()

G_elem = graph_ruggedness_de.compute_elementary_landscape(G,
                                                 50)
dir_energy = graph_ruggedness_de.compute_dirichlet_energy(G=G_elem)
values = [node[1]['value'] for node in G_elem.nodes(data=True)]
viridis = plt.cm.get_cmap('viridis', 10)
node_colors = [viridis((value - min(values)) / (max(values) - min(values))) for value in values]
nx.draw(G_elem, pos, node_color=node_colors, with_labels=False, edgecolors='black', node_size=100, width=0.75, edge_color='grey')
plt.savefig('figures/Figure_1/50th_eigvector_graph.pdf')
plt.show()

plt.figure(figsize=(1.05, 1.5))
plt.bar(x=0, height=dir_energy, color='grey')
plt.xticks([])
plt.ylim(0, 15)
plt.ylabel('Dirichlet Energy')
plt.tight_layout()
plt.savefig('figures/Figure_1/eigenvector_50_DE.pdf')
plt.show()

plt.figure(figsize=(2.5, 1.5))
plt.hist(values, bins=22, color='grey')
plt.xlabel('Simulated fitness')
plt.ylabel('Frequency')
plt.tight_layout()
plt.savefig('figures/Figure_1/eigenvector_50_hist.pdf')
plt.show()